# Pyhton Code for Calculating Human Thermal Indices (12)

## Steps:
1) All models are in one folder name ' CMIP6_Models'
2) Every Folder have 4 scenario's
    1) Historical
    2) SSP1-2.6
    3) SSP2-4.5
    4) SSP5-8.5

3) Every scenario of models have 3 '.nc' files for Near-Surface-Temperature (tas), Near-Surface-Wind-Speed (sfcWind), Near-Surafce-Raltive  Humidity (hurs). And also have one empty folder name HTI_indices.
4) Now This code will iterate through each model and all indices will be stored in their respective destinations. Progress can be seen by bar (using tqdm package)
     - HTI Indices will stores in .. Processed_Data/HTI_Indices
5) Calculation Vapour Pressure is also needed so, I first claculated Vapour pressure using Tetens Equation

    $${\displaystyle E_a = \left({\frac {RH}{100}}\right) . P } $$
    $${\displaystyle P=0.61078\exp \left({\frac {17.27T}{T+237.3}}\right)} $$

In [3]:
import xarray as xr
import numpy as np
import os
from colorama import Fore, Back, Style
from tqdm import tqdm
import math

In [4]:
data_dir = "../Raw_Data/CMIP6_Models/"
models = os.listdir(data_dir)

In [5]:
def load_cmip_data(scenario_dir):
    
    files = [ os.path.join(scenario_dir, f) for f in os.listdir(scenario_dir) if f.endswith('.nc') ]
    dataset = xr.open_mfdataset(files, decode_times=False, compat='override')
    
    # Manually decode the time coordinate
    time = dataset['time']
    decoded_time = xr.coding.times.decode_cf_datetime(time, time.units)

    # Replace the original time coordinate with the decoded time
    dataset['time'] = decoded_time

    T = dataset['tas'] - 273.15 # Convert Kelvin to Celsius
    V = dataset['sfcWind']
    RH = dataset['hurs']
    
    return T, RH, V

### Calculation of Vapour Pressure

In [7]:
def calculate_vapor_pressure(T, RH):
    # Tetens formula to calculate vapor pressure (in kPa)
    P = 0.61078 * np.exp((17.27 * T) / (T + 237.3))
    E_a = (RH / 100) * P
    return E_a

### Calculation of Indices

In [9]:
def calculate_indices(T, RH, V, E_a):
    indices = {}
    
    # Convert Celsius to Fahrenheit
    T_F = T * 9 / 5 + 32
    
    # Simple Heat Index Calculation
    HI_simple = 0.5 * (T_F + 61.0 + ((T_F - 68.0) * 1.2) + (RH * 0.094))
    HI_simple = (HI_simple + T_F) / 2
    
    # Initialize HI array
    HI = HI_simple.copy()
    
    # Full regression equation
    full_regression = -42.379 + 2.04901523 * T_F + 10.14333127 * RH - 0.22475541 * T_F * RH - 0.00683783 * T_F**2 - 0.05481717 * RH**2 + 0.00122874 * T_F**2 * RH + 0.00085282 * T_F * RH**2 - 0.00000199 * T_F**2 * RH**2
    
    # Adjustments
    low_rh_adjustment = ((13 - RH) / 4) * np.sqrt((17 - np.abs(T_F - 95)) / 17)
    high_rh_adjustment = ((RH - 85) / 10) * ((87 - T_F) / 5)
    
    # Apply full regression where HI_simple >= 80
    HI = np.where(HI_simple >= 80, full_regression, HI)
    
    # Apply low RH adjustment
    HI = np.where((RH < 13) & (T_F >= 80) & (T_F <= 112), HI - low_rh_adjustment, HI)
    
    # Apply high RH adjustment
    HI = np.where((RH > 85) & (T_F >= 80) & (T_F <= 87), HI + high_rh_adjustment, HI)
    
    # Convert HI back to Celsius
    HI_C = (HI - 32) * 5 / 9
    
    indices['HI'] = HI_C
    indices['AT_in'] = -1.3 + 0.92 * T + 2.2 * E_a
    indices['AT_out'] = -2.7 + 1.04 * T + 2 * E_a - 0.65 * V
    WBT = T * np.arctan(0.151977 * (RH + 8.313659)**0.5) + np.arctan(T + RH) - np.arctan(RH - 1.676331) + 0.00391838 * RH**1.5 * np.arctan(0.023101 * RH) - 4.686035
    indices['DI'] = 0.5 * WBT + 0.5 * T
    indices['ET'] = T - 0.4 * (T - 10) * (1 - 0.001 * RH)
    indices['HMI'] = T + 0.5555 * (0.1 * E_a - 10)
    indices['MDI'] = 0.75 * WBT + 0.38 * T
    indices['SAT'] = T
    indices['sWGBT'] = 0.567 * T + 0.0393 * E_a + 3.94
    indices['WBT'] = WBT
    indices['NET'] = 37 - ((37 - T) / ((0.68 - 0.0014 * RH) + (1 / (1.76 + 1.4 * V**0.75)))) - 0.29 * T * (1 - 0.01 * RH)
    indices['WCT'] = 13.12 + 0.6215 * T - 11.37 * (V * 3.6)**0.16 + 0.3965 * T * (V * 3.6)**0.16
    
    return indices


In [10]:
def store_indices(indices, storage_dir):
    os.makedirs(storage_dir, exist_ok=True)  # Ensure the directory exists
    
    for index_name, index_values in tqdm(indices.items()):
        index_da = xr.DataArray(index_values, dims=T.dims, coords=T.coords, name=index_name)
        index_da.to_netcdf(os.path.join(storage_dir, f'{index_name}.nc'))
        

In [11]:
# Example of custom bar format with color
bar_format = f'{Fore.GREEN}{Back.RED}{Style.BRIGHT}{{l_bar}}{{bar}}| {{n_fmt}}/{{total_fmt}} [{Fore.YELLOW}{{elapsed}}<{Fore.CYAN}{{remaining}}, {{rate_fmt}}{{postfix}}]{Style.RESET_ALL}'

In [12]:
total_models = len(models)
with tqdm(total=total_models, desc="Processing models", unit="model", bar_format=bar_format) as pbar_main:
    
    for model in models:
        model_dir = os.path.join(data_dir, model)
        scenarios = os.listdir(model_dir)
    
        total_scenarios = len(scenarios)
        with tqdm(total=total_scenarios, desc="Processing Scenarios", unit="scenario", bar_format=bar_format) as pbar:
            for scenario in scenarios:
                scenario_dir = os.path.join(model_dir, scenario)
                
                T, RH, V = load_cmip_data(scenario_dir)
                E_a = calculate_vapor_pressure(T, RH)
                print(f'Calculation of {model} {scenario} started')
                indices = calculate_indices(T, RH, V, E_a)
                print(f'Calculation of {model} {scenario} Done')
                HT_indices_dir = "../Processed_Data/HTI_Indices/"
                storage_dir = os.path.join(HT_indices_dir,model,scenario)
                store_indices(indices, storage_dir)
                pbar.update(1)
        pbar_main.update(1)

Processing Scenarios:   0%|                                                                | 0/4 [00:00<?, ?scenario/s]C:\ProgramData\anaconda3\Lib\site-packages\xarray\conventions.py:431: SerializationWarning: variable 'hurs' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
C:\ProgramData\anaconda3\Lib\site-packages\xarray\conventions.py:431: SerializationWarning: variable 'sfcWind' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
C:\ProgramData\anaconda3\Lib\site-packages\xarray\conventions.py:431: SerializationWarning: variable 'tas' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


Calculation of CESM2 (USA) Historical started


C:\ProgramData\anaconda3\Lib\site-packages\dask\core.py:127: RuntimeWarning: invalid value encountered in sqrt
  return func(*(_execute_task(a, cache) for a in args))


Calculation of CESM2 (USA) Historical Done




  0%|                                                                                           | 0/12 [00:00<?, ?it/s]

  8%|██████▉                                                                            | 1/12 [00:06<01:08,  6.27s/it]

 17%|█████████████▊                                                                     | 2/12 [00:14<01:15,  7.54s/it]

 25%|████████████████████▊                                                              | 3/12 [00:26<01:25,  9.53s/it]

 33%|███████████████████████████▋                                                       | 4/12 [00:39<01:26, 10.78s/it]

 42%|██████████████████████████████████▌                                                | 5/12 [00:48<01:12, 10.38s/it]

 50%|█████████████████████████████████████████▌                                         | 6/12 [00:57<00:58,  9.73s/it]

 58%|████████████████████████████████████████████████▍                                  | 7/12 [01:13<00:58, 11.75s/it]

 67%|█████████████████████████

...



  0%|                                                                                           | 0/12 [00:00<?, ?it/s]

  8%|██████▉                                                                            | 1/12 [00:01<00:21,  1.93s/it]

 17%|█████████████▊                                                                     | 2/12 [00:05<00:30,  3.00s/it]

 25%|████████████████████▊                                                              | 3/12 [00:10<00:35,  3.93s/it]

 33%|███████████████████████████▋                                                       | 4/12 [00:16<00:37,  4.63s/it]

 42%|██████████████████████████████████▌                                                | 5/12 [00:19<00:28,  4.04s/it]

 50%|█████████████████████████████████████████▌                                         | 6/12 [00:22<00:22,  3.82s/it]

 58%|████████████████████████████████████████████████▍                                  | 7/12 [00:28<00:21,  4.33s/it]

 67%|█████████████████████████

Calculation of CESM2 (USA) SSP5-8.5 started


C:\ProgramData\anaconda3\Lib\site-packages\dask\core.py:127: RuntimeWarning: invalid value encountered in sqrt
  return func(*(_execute_task(a, cache) for a in args))


Calculation of CESM2 (USA) SSP5-8.5 Done




  0%|                                                                                           | 0/12 [00:00<?, ?it/s]

  8%|██████▉                                                                            | 1/12 [00:03<00:35,  3.19s/it]

 17%|█████████████▊                                                                     | 2/12 [00:09<00:47,  4.73s/it]

 25%|████████████████████▊                                                              | 3/12 [00:15<00:49,  5.50s/it]

 33%|███████████████████████████▋                                                       | 4/12 [00:22<00:48,  6.09s/it]

 42%|██████████████████████████████████▌                                                | 5/12 [00:25<00:36,  5.17s/it]

 50%|█████████████████████████████████████████▌                                         | 6/12 [00:30<00:29,  4.95s/it]

 58%|████████████████████████████████████████████████▍                                  | 7/12 [00:36<00:27,  5.40s/it]

 67%|█████████████████████████

## 12 Indices Equations

| $$\text{Name}$$  | $$\text{Representation}$$ | $$\text{Expression}$$ |
|----------|----------|------------|
| $$ \text{Apparent Temperature (indoors)} $$ | $$AT_{in}$$ | $$-1.3 + 0.92 \cdot T + 2.2 \cdot E_a$$ |
| | $$AT_{out}$$ | $$-2.7 + 1.04 \cdot T + 2 \cdot E_a - 0.65 \cdot V$$ |
| | $$WBT$$ | $$T \cdot \arctan(0.151977 \cdot (\text{RH} + 8.313659)^{0.5}) + \arctan(T + \text{RH}) - \arctan(\text{RH} - 1.676331) + 0.00391838 \cdot \text{RH}^{1.5} \cdot \arctan(0.023101 \cdot \text{RH}) - 4.686035$$ |
| | $$DI$$ | $$0.5 \cdot WBT + 0.5 \cdot T$$ |
| | $$ET$$ | $$T - 0.4 \cdot (T - 10) \cdot (1 - 0.001 \cdot \text{RH})$$ |
| | $$HI$$ | $$-8.784695 + 1.61139411 \cdot T - 2.338549 \cdot \text{RH} - 0.14611605 \cdot T \cdot \text{RH} - 1.2308094e-2 \cdot T^{2} - 1.6424828e-2 \cdot \text{RH}^{2} + 2.211732e-3 \cdot T^{2} \cdot \text{RH} + 7.2546e-4 \cdot T \cdot \text{RH}^{2} + 3.582e-6 \cdot T^{2} \cdot \text{RH}^{2}$$ |
| | $$HMI$$ | $$T + 0.5555 \cdot (0.1 \cdot E_a - 10)$$ |
| | $$MDI$$ | $$0.75 \cdot WBT + 0.38 \cdot T$$ |
| $$ \text{Surface Air Temperature}$$ | $$SAT$$ | $$T$$ |
| | $$sWGBT$$ | $$0.567 \cdot T + 0.0393 \cdot E_a + 3.94$$ |
| | $$WBT$$ | $$WBT$$ |
| | $$NET$$ | $$37 - ((37 - T) / ((0.68 - 0.0014 \cdot RH) + (1 / (1.76 + 1.4 \cdot V^{0.75}))))) - 0.29 \cdot T \cdot (1 - 0.01 \cdot RH)$$ |
| | $$WCT$$ | $$13.12 + 0.6215 \cdot T - 11.37 \cdot (V^{3/2})^{0.16} + 0.3965 \cdot T^{1/16}$$ |

$$ \text{Equations of the 12 thermal indices. } T \text{ is air temperature (°C)}  \text{ at 2-meter height, }$$ $$\text{ RH  is relative humidity } \text {(%), } \text{ V is wind speed (m/s), } \text{and }E_a \text{ is actual vapor pressure (kPa). } $$ $$ \text { The unit of all human thermal indices is degree Celsius (°C).} $$